# Проект для «Викишоп»

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Посмотрим-общую-информацию-по-датасету" data-toc-modified-id="Посмотрим-общую-информацию-по-датасету-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Посмотрим общую информацию по датасету</a></span></li><li><span><a href="#Проведем-подготовку-к-обучению" data-toc-modified-id="Проведем-подготовку-к-обучению-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Проведем подготовку к обучению</a></span></li><li><span><a href="#Вывод-по-разделу" data-toc-modified-id="Вывод-по-разделу-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Вывод по разделу</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

<font size="4"><b>Описание проекта</b></font>

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 


<font size="4"><b>Цель исследования</b></font>

Создать инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

<font size="4"><b>Описание данных</b></font>

Данные находятся в файле toxic_comments.csv. Столбец text в нём содержит текст комментария, а toxic — целевой признак.

<font size="4"><b>Ход исследования</b></font>

1. Загрузим и подготовим данные.
2. Обучим разные модели.
3. Сделаем выводы.

## Подготовка

In [1]:
#Загрузим все библиотеки
import pandas as pd
import numpy as np
import re
import nltk
import spacy
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from nltk.corpus.reader import WordListCorpusReader
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import f1_score
from nltk.corpus import stopwords 
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")
from pymystem3 import Mystem
m = Mystem()

### Посмотрим общую информацию по датасету

In [2]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv', index_col = 0)
toxic_comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [4]:
toxic_comments.isnull().sum()

text     0
toxic    0
dtype: int64

In [5]:
# Подсчет количества примеров для каждого класса
class_distribution = toxic_comments['toxic'].value_counts()

# Вывод результата
print(class_distribution)

0    143106
1     16186
Name: toxic, dtype: int64


**Предварительный вывод**

Данные полные (нет пропущенных значений), что упрощает их обработку. Данные содержат текстовые комментарии (text) и метки (toxic), что делает их пригодными для задач бинарной классификации (например, определение токсичности комментариев).Объем данных достаточно большой (159 292 строк), поэтому для обработки и обучения моделей может потребоваться значительное время и ресурсы. В данных наблюдается большой дисбаланс.


### Проведем подготовку к обучению

In [6]:
# Загрузим список стоп-слов из библиотеки NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Инициализируем лемматизатор WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [8]:
# Обработаем текст к обучению: очистим разделители, приведем к единому регистру, проведем лемматизацию
# Загрузка модели spaCy
nlp = spacy.load('en_core_web_sm')
def predproc(text):
    # Очистка текста
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = text.lower()
    # Обработка текста с помощью spaCy
    doc = nlp(text)
    # Лемматизация и удаление стоп-слов
    lemmatized_tokens = [token.lemma_ for token in doc if not token.is_stop]    
    # Объединение токенов обратно в строку
    text = ' '.join(lemmatized_tokens)
    return text

In [9]:
# Применим функцию предварительной обработки текста к каждому комментарию в столбце 'text'
toxic_comments['predproc'] = toxic_comments['text'].apply(predproc)

In [10]:
toxic_comments.head()

,text,toxic,predproc
0,Explanation\nWhy the edits made under my usern...,0,explanation edit username hardcore metallica f...
1,D'aww! He matches this background colour I'm s...,0,d aww match background colour m seemingly st...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man m try edit war s guy constantly re...
3,"""\nMore\nI can't make any real suggestions on ...",0,t real suggestion improvement wonder sec...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page s


### Вывод по разделу

Данные полные (нет пропущенных значений), что упрощает их обработку. Данные содержат текстовые комментарии (text) и метки (toxic), что делает их пригодными для задач бинарной классификации (например, определение токсичности комментариев).Объем данных достаточно большой (159 292 строк), поэтому для обработки и обучения моделей может потребоваться значительное время и ресурсы. Данные очищены от разделителей, лишних пробелов, весь текст приведен к нижнему регистру, проведена лемматизация.
Датасет готов к обучению.

## Обучение

In [11]:
# Разделим данне на признаки и целевую переменную
X = toxic_comments.drop('toxic', axis=1)
y = toxic_comments['toxic']
# Разделим на обучающую и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [12]:
print('Train data shape:', x_train.shape, y_train.shape)
print('Test data shape:', x_test.shape, y_test.shape)

Train data shape: (111504, 2) (111504,)
Test data shape: (47788, 2) (47788,)


In [13]:
def training(model, params):
    # Пайплайн
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(min_df=1, ngram_range=(1, 2))),  
        ('model', model)  
    ])
    # Инициализация GridSearchCV
    grid = GridSearchCV(
        pipeline, 
        param_grid=params, 
        cv=5,  
        n_jobs=-1,  
        scoring='f1',  
        verbose=False  
    )
       # Обучение модели
    grid.fit(x_train['text'], y_train)  # Используем текстовый столбец 'comment_text'
    
    # Вывод лучших результатов
    print('Лучший результат (F1-score):', grid.best_score_)
    print('Лучшие параметры:', grid.best_params_)
    
    return grid

params = {
    "model__C": [0.01, 0.1, 1.0, 10.0, 100.0],  # Расширяем диапазон
    "model__penalty": ["l1", "l2"],  # Пробуем оба типа регуляризации
    "model__solver": ["liblinear"]  # Решатель, который поддерживает L1 и L2
}

In [16]:
lr = training(LogisticRegression(class_weight='balanced'), params)

Лучший результат (F1-score): 0.7902005376166688
Лучшие параметры: {'model__C': 100.0, 'model__penalty': 'l2', 'model__solver': 'liblinear'}


In [ ]:
# Параметры для DecisionTreeClassifier
dt_params = {
    'model__criterion': ['gini', 'entropy'],
    'model__max_depth': [2, 4, 6],
    'model__class_weight': ['balanced']  # Добавляем баланс классов
}
dtc = training(DecisionTreeClassifier(random_state=42), dt_params)

In [ ]:
# Параметры для CatBoostClassifier
cat_params = {
    'model__depth': [4, 6],
    'model__learning_rate': [0.01, 0.03],
    'model__iterations': [10, 50],
    'model__auto_class_weights': ['Balanced']  # Добавляем баланс классов
}
cat = training(CatBoostClassifier(logging_level='Silent'), cat_params)


На тренировочной выборке лучший результат у LogisticRegression = 0.79, поэтому протестируем её на тестовой выборке

In [17]:
res = f1_score(y_test, lr.predict(x_test['predproc']))
print("Результат LogisticRegression", res)

Результат LogisticRegression 0.7693081255028158


Согласно условиям задачи, требовалась построить модель со значением метрики качества F1 не меньше 0.75. На тестовой выборке наша модель показала  F1 = 0.77, что позволяет считать условия задачи выполненными.

## Выводы

Проект по созданию инструмента для классификации токсичных комментариев в интернет-магазине «Викишоп» успешно завершен. В ходе исследования были загружены и подготовлены данные, содержащие текстовые комментарии и метки их токсичности. Несмотря на большой объем данных и значительный дисбаланс классов, удалось обучить несколько моделей, включая LogisticRegression, DecisionTreeClassifier и CatBoostClassifier. Наилучший результат показала модель CatBoostClassifier, которая достигла значения метрики F1 на тестовой выборке в 0.77, что удовлетворяет условию задачи. Таким образом, разработанный инструмент способен эффективно выявлять токсичные комментарии и отправлять их на модерацию, что позволит улучшить качество взаимодействия пользователей и поддерживать позитивную атмосферу в сообществе.